# Preparing the example dataset for use with QUANTS

Our example dataset is raw counts from the BRCA1 saturation genome editing (SGE) experiment by Findlay *et al.*. 

> Findlay, G.M., Daza, R.M., Martin, B. *et al.*   
> **Accurate classification of BRCA1 variants with saturation genome editing.**  
> *Nature 562, 217–222 (2018).*   
> [https://doi.org/10.1038/s41586-018-0461-z](https://doi.org/10.1038/s41586-018-0461-z)

For this example we will be quantifying multiple samples which correspond to BRCA1 exon 2:

* plasmid
* negative control
* day 5 (2 replicates)
* day 11 (2 replicates)

***

## Loading R library dependencies

Before we start, we need to load R library dependencies. These are only required for this example, they are not a dependency of QUANTS.

In [1]:
library(tidyverse)
library(readxl)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



***

## Downloading raw sequencing data from Findlay screens

To download the Findlay SRA FASTQs we used [SRA toolkit](https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/2.11.2/sratoolkit.2.11.2-ubuntu64.tar.gz) version 2.11.2 following the instructions [here](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit).

Here we loop over the following SRA run ids from [PRJNA481326](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA481326) which correspond to BRCA1 exon 2 day 5 and day 11 samples, plasmid library and negative control:

* **SRR7525820** - X2-lib (library)  
* **SRR7525821** - X2-neg (negative control)  
* **SRR7525822** - X2rL41-post (Day 11 gDNA sample)  
* **SRR7525823** - X2rL41-pre (Day 5 gDNA sample)  
* **SRR7525825** - X2rL42-post (Day 11 gDNA sample)  
* **SRR7525826** - X2rL42-pre (Day 5 gDNA sample)  

In [2]:
for (run in c('SRR7525820', 'SRR7525821', 'SRR7525822', 'SRR7525823', 'SRR7525825', 'SRR7525826'))  {
    fastq_dump_cmd <- paste("fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+'", run)
    print(fastq_dump_cmd)
    if (!file.exists(paste0('example_input/fastq/', run, '_1.fastq.gz')) || !file.exists(paste0('example_input/fastq/', run, '_2.fastq.gz'))) {
        system(fastq_dump_cmd)   
    }
}

[1] "fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+' SRR7525820"
[1] "fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+' SRR7525821"
[1] "fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+' SRR7525822"
[1] "fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+' SRR7525823"
[1] "fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+' SRR7525825"
[1] "fastq-dump --split-files --gzip --outdir example_input/fastq --defline-seq '@$ac.$si.$sg/$ri' --defline-qual '+' SRR7525826"


***

## Downloading Findlay et al. raw counts 

Raw counts, functional scores and annotations from the BRCA1 SGE experiments can be found in [Supplementary Table 1](https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-018-0461-z/MediaObjects/41586_2018_461_MOESM3_ESM.xlsx). 

In [3]:
# URL of supplementary table 1 from Findlay et al. which contains variant positions and counts
findlay_suppl1_url <- 'https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-018-0461-z/MediaObjects/41586_2018_461_MOESM3_ESM.xlsx'

# Download supplementary table and read XLSX sheet 1 into a data frame skipping the first two rows (unwanted headers) 
tmpfile <- tempfile()
download.file(url = findlay_suppl1_url, destfile = tmpfile, quiet = TRUE)
findlay_suppl1 <- suppressWarnings(readxl::read_xlsx(tmpfile, sheet = 1, skip = 2))
unlink(tmpfile)

# Show data frame
head(findlay_suppl1)

gene,chromosome,position (hg19),reference,alt,transcript_ID,transcript_position,transcript_ref,transcript_alt,transcript_variant,⋯,aGVGD.diff,aGVGD.class,clinvar,clinvar_simple,gnomAD_AF,bravo_AF,flossies_AF,WT.HAP1.function.score.r1,WT.HAP1.function.score.r2,WT.HAP1.function.score.mean
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
BRCA1,17,41276135,T,G,NM_007294.3,-19-3,A,C,c.-19-3A>C,⋯,NA,NA,absent,absent,0.0000000,0.00000000,0,0.36944644087971701,0.259903001034893,0.31467472095730498
BRCA1,17,41276135,T,C,NM_007294.3,-19-3,A,G,c.-19-3A>G,⋯,NA,NA,Conflicting interpretations of pathogenicity,Conflicting interpretations of pathogenicity,0.0001275,0.00016724,0,-0.33863672461542499,0.30135564822599198,-1.86405381947163E-2
BRCA1,17,41276135,T,A,NM_007294.3,-19-3,A,T,c.-19-3A>T,⋯,NA,NA,absent,absent,0.0000000,0.00000000,0,-0.97517572868796798,-0.10121785206809,-0.53819679037802803
BRCA1,17,41276134,T,G,NM_007294.3,-19-2,A,C,c.-19-2A>C,⋯,NA,NA,absent,absent,0.0000000,0.00000000,0,-1.76677702938779,0.52989154772828595,-0.61844274082975403
BRCA1,17,41276134,T,C,NM_007294.3,-19-2,A,G,c.-19-2A>G,⋯,NA,NA,Pathogenic,Pathogenic,0.0000000,0.00000000,0,NA,NA,NA
BRCA1,17,41276134,T,A,NM_007294.3,-19-2,A,T,c.-19-2A>T,⋯,NA,NA,absent,absent,0.0000000,0.00000000,0,0.74115354036225101,-6.5988050752730204E-2,0.33758274480476003


In [4]:
# Extract hg19 annotations and raw count columns from supplementary data
findlay_raw_counts <- findlay_suppl1 %>% 
    select(chromosome, 'position_hg19' = 'position (hg19)', reference, alt, library, negative, d5.r1, d11.r1, d5.r2, d11.r2)

# Show data frame
head(findlay_raw_counts)

chromosome,position_hg19,reference,alt,library,negative,d5.r1,d11.r1,d5.r2,d11.r2
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17,41276135,T,G,546,1,887,903,705,968
17,41276135,T,C,595,1,929,1139,759,1516
17,41276135,T,A,1028,2,1448,1841,1532,2578
17,41276134,T,G,705,1,950,1226,1203,1306
17,41276134,T,C,832,3,1552,1259,1153,1546
17,41276134,T,A,796,2,1208,1583,1056,1216


Because the VaLiAnT example library was built against GRCh38 and the Findlay *et al.* variants are annotated to hg19, we need to perform a lift over to get the corresponding coordinates. This was done using the [hgLiftOver](https://genome.ucsc.edu/cgi-bin/hgLiftOver) tool from UCSC and saved in [hg19_grch38_liftover.tsv](hg19_grch38_liftover.tsv).

In [5]:
# Read in lift over coordinates
lift_over_coords <- read.delim('hg19_grch38_liftover.tsv', sep = "\t", header = T)

# Add lift over coordinates to raw counts
findlay_raw_counts <- findlay_raw_counts %>%
    left_join(lift_over_coords, by = 'position_hg19') %>%
    relocate(position_GRCh38, .after = 'position_hg19')

# Show data frame
head(findlay_raw_counts)

chromosome,position_hg19,position_GRCh38,reference,alt,library,negative,d5.r1,d11.r1,d5.r2,d11.r2
<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17,41276135,43124118,T,G,546,1,887,903,705,968
17,41276135,43124118,T,G,546,1,887,903,705,968
17,41276135,43124118,T,G,546,1,887,903,705,968
17,41276135,43124118,T,C,595,1,929,1139,759,1516
17,41276135,43124118,T,C,595,1,929,1139,759,1516
17,41276135,43124118,T,C,595,1,929,1139,759,1516


***

## Downloading VaLiAnT oligo library

As QUANTS requires a library, we will use [VaLiAnT](https://github.com/cancerit/VaLiAnT) to generate a library corresponding to BRCA1 exon2:

> Barbon, L., Offord, V., Radford, E.J., *et al.*  
> **Variant Library Annotation Tool (VaLiAnT): an oligonucleotide library design and annotation tool for saturation genome editing and other deep mutational scanning experiments.**   
> *Bioinformatics 38(4), 892–899 (2022).*. 
> https://doi.org/10.1093/bioinformatics/btab776

[VaLiAnT](https://github.com/cancerit/VaLiAnT) is an oligonucleotide library design and annotation tool for SGE and other deep mutational scanning experiments. See [example_input/valiant_library/README.md](example_input/valiant_library/README.md) for details of creating a VaLiAnT library corresponding to BRAC1 exon 2 coding sequence (CDS).

In [6]:
brca1_exon2_valiant_lib_file <- "example_input/valiant_library/brca1_nuc_output/chr17_43123941_43124175_minus_sgRNA_exon2_meta.csv"

# Read VaLiAnT library into data frame
brca1_exon2_valiant_lib <- read.delim(brca1_exon2_valiant_lib_file, sep = ",", header = T, check.names = F)

# Show data frame
head(brca1_exon2_valiant_lib)

,oligo_name,species,assembly,gene_id,transcript_id,src_type,ref_chr,ref_strand,ref_start,ref_end,⋯,vcf_var_id,mut_position,ref,new,ref_aa,alt_aa,mut_type,mutator,oligo_length,mseq
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,ENST00000357654.9.ENSG00000012048.23_chr17:43124017_C>A_snv_rc,homo sapiens,GRCh38,ENSG00000012048.23,ENST00000357654.9,ref,chr17,-,43123941,43124175,⋯,NA,43124017,C,A,C,F,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTTGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
2,ENST00000357654.9.ENSG00000012048.23_chr17:43124017_C>T_snv_rc,homo sapiens,GRCh38,ENSG00000012048.23,ENST00000357654.9,ref,chr17,-,43123941,43124175,⋯,NA,43124017,C,T,C,Y,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTAGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
3,ENST00000357654.9.ENSG00000012048.23_chr17:43124017_C>G_snv_rc,homo sapiens,GRCh38,ENSG00000012048.23,ENST00000357654.9,ref,chr17,-,43123941,43124175,⋯,NA,43124017,C,G,C,S,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTCGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
4,ENST00000357654.9.ENSG00000012048.23_chr17:43124018_A>C_snv_rc,homo sapiens,GRCh38,ENSG00000012048.23,ENST00000357654.9,ref,chr17,-,43123941,43124175,⋯,NA,43124018,A,C,C,G,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCGGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
5,ENST00000357654.9.ENSG00000012048.23_chr17:43124018_A>T_snv_rc,homo sapiens,GRCh38,ENSG00000012048.23,ENST00000357654.9,ref,chr17,-,43123941,43124175,⋯,NA,43124018,A,T,C,S,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCAGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
6,ENST00000357654.9.ENSG00000012048.23_chr17:43124018_A>G_snv_rc,homo sapiens,GRCh38,ENSG00000012048.23,ENST00000357654.9,ref,chr17,-,43123941,43124175,⋯,NA,43124018,A,G,C,R,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCCGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC


In Findlay et al. the region targeted is much smaller than the window generated here (GRCh38 e106 17:43124013-43124118). We also produce a shorter oligo sequence for improved comparability.

***

## Combining VaLiAnT oligos and Findlay raw counts

We can then combine the raw counts and annotation from Findlay *et al.* with the VaLiAnT oligos and their annotations. 

In [7]:
# Combine raw counts with VaLiAnT oligos using lift over coordinates and ref/alt annotations
combined_annotations <- findlay_raw_counts %>% 
    filter(position_GRCh38 %in% brca1_exon2_valiant_lib$mut_position) %>%
    left_join(brca1_exon2_valiant_lib, by = c('position_GRCh38' = 'mut_position', 'reference' = 'ref', 'alt' = 'new')) %>%
    unique()

# Write combined annotations to file
write.table(combined_annotations, 'combined_findlay_valiant_annotations_and_counts.tsv', sep = "\t", row.names = F, quote = F)

# Show data frame
head(combined_annotations)

chromosome,position_hg19,position_GRCh38,reference,alt,library,negative,d5.r1,d11.r1,d5.r2,⋯,ref_seq,pam_seq,vcf_alias,vcf_var_id,ref_aa,alt_aa,mut_type,mutator,oligo_length,mseq
<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
17,41276113,43124096,T,G,828,2,685,663,847,⋯,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATGGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAATGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATCGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAGTGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,NA,NA,M,L,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAACTGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
17,41276113,43124096,T,C,1229,1,1335,847,1179,⋯,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATGGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAATGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATCGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAGTGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,NA,NA,M,V,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAGTGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
17,41276113,43124096,T,A,867,2,658,652,701,⋯,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATGGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAATGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATCGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAGTGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,NA,NA,M,L,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAATTGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
17,41276112,43124095,A,T,681,1,641,391,927,⋯,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATGGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAATGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATCGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAGTGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,NA,NA,M,K,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAAAGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC
17,41276112,43124095,A,G,797,1,756,670,810,⋯,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATGGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAATGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,GTAAGGTCAATTCTGTTCATTTGCATAGGAGATAATCATAGGAATCCCAAATTAATACACTCTTGTGCTGACTTACCAGATCGGACACTCTAAGATTTTCTGCATAGCATTAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCATTTCTTTCTGTTCCAGTGAACTTTAACACATTAGAAAAACATATATATATATCTTTTTAAAAGGTTTATAAAATGACAAC,NA,NA,M,T,mis,snv,235,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTA

***

## Convert VaLiAnT library for use with pyCROQUET

[QUANTS](https://github.com/cancerit/QUANTS) is a Nextflow pipeline which is comprised of several stages which leverage existing software. Quantification of oligo abundance is performed by [pyCROQUET](https://github.com/cancerit/pycroquet) which has a pre-defined library format which is described in the [pyCROQUET wiki](https://github.com/cancerit/pycroquet/wiki/Guide-library-format).

In [8]:
# Pull relevant columns from combined annotations into pyCROQUET format
pycroquet_lib <- combined_annotations %>%
    select('id' = oligo_name, 'sgrna_ids' = oligo_name, 'sgrna_seqs' = mseq) %>%
    mutate('gene_pair_id' = 'BRCA1_exon2_CDS')

# Show data frame
head(pycroquet_lib)

id,sgrna_ids,sgrna_seqs,gene_pair_id
<chr>,<chr>,<chr>,<chr>
ENST00000357654.9.ENSG00000012048.23_chr17:43124096_T>G_snv_rc,ENST00000357654.9.ENSG00000012048.23_chr17:43124096_T>G_snv_rc,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAACTGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC,BRCA1_exon2_CDS
ENST00000357654.9.ENSG00000012048.23_chr17:43124096_T>C_snv_rc,ENST00000357654.9.ENSG00000012048.23_chr17:43124096_T>C_snv_rc,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAGTGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC,BRCA1_exon2_CDS
ENST00000357654.9.ENSG00000012048.23_chr17:43124096_T>A_snv_rc,ENST00000357654.9.ENSG00000012048.23_chr17:43124096_T>A_snv_rc,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAATTGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC,BRCA1_exon2_CDS
ENST00000357654.9.ENSG00000012048.23_chr17:43124095_A>T_snv_rc,ENST00000357654.9.ENSG00000012048.23_chr17:43124095_A>T_snv_rc,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAAAGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC,BRCA1_exon2_CDS
ENST00000357654.9.ENSG00000012048.23_chr17:43124095_A>G_snv_rc,ENST00000357654.9.ENSG00000012048.23_chr17:43124095_A>G_snv_rc,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAACGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC,BRCA1_exon2_CDS
ENST00000357654.9.ENSG00000012048.23_chr17:43124095_A>C_snv_rc,ENST00000357654.9.ENSG00000012048.23_chr17:43124095_A>C_snv_rc,GTTGTCATTTTATAAACCTTTTAAAAAGATATATATATATGTTTTTCTAATGTGTTAAAGTTCACTGGAACAGAAAGAAAGGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGGTAAGTCAGCACAAGAGTGTATTAATTTGGGATTCCTATGATTATCTCCTATGCAAATGAACAGAATTGACCTTAC,BRCA1_exon2_CDS


In [9]:
# Set pyCROQUET library file path
pycroquet_lib_file <- "example_input/BRCA1_exon2_pyCROQUET.tsv"

# Write pyCROQUET headers
write(paste0("##library-type: single"), file = pycroquet_lib_file)
write(paste0("##library-name: ", 'BRCA1_exon2_CDS'), file = pycroquet_lib_file, append = TRUE)
write(paste("#id", "sgrna_ids", "sgrna_seqs", "gene_pair_id", sep = "\t"), file = pycroquet_lib_file, append = TRUE)

# Write oligos 
for (i in 1:nrow(pycroquet_lib)) {
    write(paste(pycroquet_lib$id[i], 
                pycroquet_lib$sgrna_ids[i], 
                pycroquet_lib$sgrna_seqs[i], 
                pycroquet_lib$gene_pair_id[i], 
                sep = "\t" ), 
          file = pycroquet_lib_file, append = TRUE)
}